In [4]:
# Start managing our Data
def select_products(df,product,sexe, mark): 
    if product == "all" and sexe == "all" and mark == "all":
        df_res = df
    elif product == "all":
        df_res = df[(df['sexe'] == sexe) & (df['mark'] == mark) ]
    elif sexe == "all":
        df_res = df[(df['product'] == product) & (df['mark'] == mark) ]
    elif mark == "all":
        df_res = df[(df['product'] == product) & (df['sexe'] == sexe) ]
        
    elif product == "all" and sexe == "all":
        df_res = df[(df['mark'] == mark) ]
    elif product == "all" and mark == "all":
        df_res = df[(df['sexe'] == sexe) ]
        
    elif sexe == "all" and mark == "all":
        df_res = df[(df['product'] == product) ]
    else :
        df_res = df[ (df['product'] == product) & (df['sexe'] == sexe) & (df['mark'] == mark) ]
        
    return df_res

selected_df = select_products(df,"chemise","homme", "all")
len(selected_df)



1923

In [5]:
# Vision générale basée sur la comparaison liniaire entre (finalprice,originprice,promo)
def give_infos(selected_df):
    return selected_df.quantile([0,.25, .5, .75 ,.99],interpolation='lower')

give_infos(selected_df)

,finalprice,originprice,promo
0.00,10.50,11.99,0.0
0.25,34.95,39.95,0.0
0.50,47.99,54.95,0.0
0.75,69.99,79.99,15.0
0.99,159.95,169.90,58.0


In [6]:
# les 5 moins cher 
def select_moins_cher(selected_df):
    moins_cher  = selected_df.sort_values(by=['finalprice'], ascending = True).head(5)
    return moins_cher

select_moins_cher(selected_df)

,product,picture,sexe,mark,description,finalprice,originprice,promo
25756,chemise,https://img01.ztat.net/article/spp-media-p1/ea...,homme,Urban,VISCOSE CAMP SHIRT - Chemise - softseagrass,10.50,34.99,70.0
25456,chemise,https://img01.ztat.net/article/spp-media-p1/92...,homme,Jack,JORBEST CHECK - Chemise - navy blazer,11.99,24.99,52.0
25445,chemise,https://img01.ztat.net/article/spp-media-p1/85...,homme,Jack,JORBEST SOLID - Chemise - navy blazer,11.99,24.99,52.0
25630,chemise,https://img01.ztat.net/article/spp-media-p1/c8...,homme,Jack,JORBEST SOLID - Chemise - forest night,11.99,11.99,0.0
25631,chemise,https://img01.ztat.net/article/spp-media-p1/d5...,homme,Jack,JORBEST SOLID - Chemise - peyote,11.99,24.99,52.0


In [7]:

# les 5 plus cher 
def select_plus_cher(selected_df):
    plus_cher  = selected_df.sort_values(by=['finalprice'], ascending = False).head(5)
    return plus_cher

select_plus_cher(selected_df)

,product,picture,sexe,mark,description,finalprice,originprice,promo
26788,chemise,https://img01.ztat.net/article/spp-media-p1/22...,homme,Paul,TAILORED - Chemise - dark blue,375.0,375.0,0.0
26027,chemise,https://img01.ztat.net/article/spp-media-p1/72...,homme,YOUNG,ADAN SOFT - Chemise - black,325.0,325.0,0.0
26929,chemise,https://img01.ztat.net/article/spp-media-p1/f3...,homme,Paul,Chemise - dark blue,290.0,290.0,0.0
25611,chemise,https://img01.ztat.net/article/spp-media-p1/96...,homme,C.P.,Pullover - rosso,281.0,419.0,33.0
26695,chemise,https://img01.ztat.net/article/spp-media-p1/1e...,homme,Vivienne,GHOST SHIRT UNISEX - Chemise - logomania,270.0,270.0,0.0


In [8]:
def select_moyens_products(selected_df):
    # Les 5 produits qui representent le milieu de l'ensemble des produits

    # Let’s take a look at the different parameters that the Pandas quantile method offers. The default arguments are provided in square [] brackets.
    # q=[0.5]: a float or an array that provides the value(s) of quantiles to calculate
    # axis=[0]: the axis to calculate the percentiles on (0 for row-wise and 1 for column-wise)
    # numeric_only=[True]: is set to False, calculate the values for datetime and timedelta columns as well
    # interpolation=['linear']: if quantiles exist between two values, how to interpolate the values

    mean_exact = selected_df[selected_df.finalprice == selected_df.quantile(
        q=0.5,                      # The percentile to calculate
        axis=0,                     # The axis to calculate the percentile on
        numeric_only=True,          # To calculate only for numeric columns
        interpolation='lower'      # The type of interpolation to use when the quantile is between 2 values
    ).finalprice].head(3)
    
    return mean_exact


select_moyens_products(selected_df)

,product,picture,sexe,mark,description,finalprice,originprice,promo
25808,chemise,https://img01.ztat.net/article/spp-media-p1/bc...,homme,Selected,SLHSLIMETHAN CUT AWAY - Chemise classique - br...,47.99,47.99,0.0
25936,chemise,https://img01.ztat.net/article/spp-media-p1/3d...,homme,Only,ONSTRACK LIFE SHIRT - Chemise - silver,47.99,47.99,0.0
26049,chemise,https://img01.ztat.net/article/spp-media-p1/bc...,homme,Only,ONSTRACK LIFE SHIRT - Chemise - black,47.99,47.99,0.0


In [10]:

def propose_best_products(selected_df):
    # Proposition de 5 produits qui ont un final price moin ou égale de la moyenne 
    # avec meilleur origin price ou avec meilleur promo 
    q_mean = selected_df.quantile(
        q = 0.5,                      # The percentile to calculate
        axis=0,                     # The axis to calculate the percentile on
        numeric_only=True,          # To calculate only for numeric columns
        interpolation='linear'      # The type of interpolation to use when the quantile is between 2 values
                                )
    
    mean_moins = selected_df[selected_df.finalprice <= q_mean.finalprice]
    mean_moins = mean_moins.sort_values(by=['originprice','promo'], ascending = False).head(10)
    best_products = pd.concat([select_moyens_products(selected_df),mean_moins])
    best_products = best_products.sort_values(by=['originprice','promo'], ascending = False).head(5)
    return best_products

propose_best_products(selected_df)


,product,picture,sexe,mark,description,finalprice,originprice,promo
25926,chemise,https://img01.ztat.net/article/spp-media-p1/0f...,homme,Jimmy,SLIM FIT - Chemise - white turquoise,46.90,109.90,57.0
26343,chemise,https://img01.ztat.net/article/spp-media-p1/7c...,homme,Basics,MIKE - Chemise - red,39.90,99.90,60.0
26632,chemise,https://img01.ztat.net/article/spp-media-p1/51...,homme,Pepe,FRAN - Chemise - truffle,46.95,94.95,51.0
26000,chemise,https://img01.ztat.net/article/spp-media-p1/41...,homme,AC&CO,Chemise - navy blue,34.99,89.99,61.0
25913,chemise,https://img01.ztat.net/article/spp-media-p1/d8...,homme,Denim,KEITH - Chemise - blau,34.95,89.95,61.0
